# Cluster Training

DOES NOT WORK YET

Issues with distributed training, also not sure if dataset class will work. 

In [1]:
import numpy as np, pandas as pd
import requests, io, os, datetime, re
from tensorflow_snowflake_class import SnowflakeDataset

import tensorflow as tf
import keras
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses, Model
from keras.applications import resnet50
from keras.preprocessing import image
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
import dask

import time

import tensorflow as tf
print(tf.__version__)

/srv/conda/envs/saturn/lib/python3.9/site-packages/snowflake/connector/options.py:94: UserWarning: You have an incompatible version of 'pyarrow' installed (4.0.1), please install a version that adheres to: 'pyarrow<3.1.0,>=3.0.0; extra == "pandas"'
  warn_incompatible_dep(


2.4.1


In [5]:
from dask.distributed import get_worker
import json 
from tensorflow.distribute import cluster_resolver, MultiWorkerMirroredStrategy, experimental
import wandb
from wandb.keras import WandbCallback

In [4]:
from dask_saturn import SaturnCluster
from dask.distributed import Client
import s3fs
import re

cluster = SaturnCluster()
client = Client(cluster)
client.wait_for_workers(3)
client

INFO:dask-saturn:Cluster is ready
INFO:dask-saturn:Registering default plugins
INFO:dask-saturn:Success!


Client Scheduler: tcp://d-steph-tensorflow-dev-c684c6ee502c425a8142c741e3f0af9a.main-namespace:8786 Dashboard: https://d-steph-tensorflow-dev-c684c6ee502c425a8142c741e3f0af9a.internal.saturnenterprise.io,Cluster Workers: 5 Cores: 80 Memory: 295.69 GiB


# Setting up worker organization

These functions create the "TF_CONFIG" env var.

In [6]:
def f():
    worker = get_worker()  # The worker on which this task is running
    return worker.address

def make_assignments(future):
    clusterset = {'worker': list(future)}
    task_assignments = []
    for i in range(len(clusterset['worker'])):
        
        if len(clusterset['worker']) < 2:
            print("Not enough workers")
        elif i == 0:
            assign = {"worker": clusterset['worker'][i], "type": "worker", "index": i}
        else:
            assign = {"worker": clusterset['worker'][i], "type": "worker", "index": i}

        task_assignments.append(assign)
        
    return {"clusterset":clusterset, "assignments":task_assignments}

def make_config(clusterset, assignments):
    worker = get_worker()
    
    #worker = clusterset['worker'][1]
    assignment = [x for x in assignments if x['worker'] == worker.address]
    results = json.dumps({
        'cluster': clusterset,
        'task': {'type': assignment[0]['type'], 'index': assignment[0]['index']}
        })
    
    os.environ['TF_CONFIG'] = results
    return(os.environ['TF_CONFIG'])

# Process Data

In this training loop, I'm trying to instruct it to use NCCL but somehow it still seems to be using gRPC.
https://www.tensorflow.org/tutorials/distribute/multi_worker_with_keras#choose_the_right_strategy


In [19]:
def train_model_fit(n_epochs, base_lr, batchsize):
    
    # --------- Start wandb --------- #
    wandb.init(entity='skirmer', project='tensorflow_rnd')
    config=wbargs

    communication_options = experimental.CommunicationOptions(
        implementation=experimental.CommunicationImplementation.NCCL)
    
    strategy = MultiWorkerMirroredStrategy(
        communication_options=communication_options,
       cluster_resolver = cluster_resolver.TFConfigClusterResolver()
   )

    with strategy.scope():
        model = tf.keras.applications.ResNet50(
            include_top=True,
            weights=None,
            classes=10)
        
        optimizer = keras.optimizers.Adam(lr=base_lr)
        model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    start = time.time()
    
    model.fit(dist_dataset,
        steps_per_epoch=96,
        epochs=n_epochs,
        callbacks=[WandbCallback()])
    
    end = time.time()-start
    print("model training time", end)
    wandb.log({"training_time":end})

    Close your wandb run 
    wandb.run.finish()

In [20]:
client.restart() # Clears memory on cluster- optional but recommended.

Client Scheduler: tcp://d-steph-tensorflow-dev-c684c6ee502c425a8142c741e3f0af9a.main-namespace:8786 Dashboard: https://d-steph-tensorflow-dev-c684c6ee502c425a8142c741e3f0af9a.internal.saturnenterprise.io,Cluster Workers: 5 Cores: 80 Memory: 295.69 GiB


In [21]:
# Prepare workers for parallel training- make the env vars on each worker.

f1 = client.run(f)  
assignments = make_assignments(f1)
worker_assign = client.run(make_config, **assignments)


In [22]:
# Dataset creation - how best to do on workers? Can we figure out a way to pull from S3?

train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/home/jovyan/git-repos/tensorflow_ds/data/clothing-dataset-small/train',
  seed=123,
  image_size=(224,224),
  batch_size=32).prefetch(2)

Found 3068 files belonging to 10 classes.


In [23]:
# Tests to see how the communication settings look

communication_options = experimental.CommunicationOptions(
    implementation=experimental.CommunicationImplementation.NCCL)

strategy = MultiWorkerMirroredStrategy(
    communication_options=communication_options,
    cluster_resolver = cluster_resolver.TFConfigClusterResolver()
)
    
dist_dataset = strategy.experimental_distribute_dataset(train_ds)


INFO:tensorflow:Using MirroredStrategy with devices ('/device:GPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/device:GPU:0',)


INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:GPU:0',), communication = CommunicationImplementation.NCCL


INFO:tensorflow:Single-worker MultiWorkerMirroredStrategy with local_devices = ('/device:GPU:0',), communication = CommunicationImplementation.NCCL


In [24]:
dist_dataset

In [25]:
# Check how the worker env vars turned out

worker_assign

{'tcp://192.168.122.132:33355': '{"cluster": {"worker": ["tcp://192.168.122.132:33355", "tcp://192.168.124.4:34529", "tcp://192.168.157.4:43091", "tcp://192.168.192.4:42475", "tcp://192.168.208.132:43737"]}, "task": {"type": "worker", "index": 0}}',
 'tcp://192.168.124.4:34529': '{"cluster": {"worker": ["tcp://192.168.122.132:33355", "tcp://192.168.124.4:34529", "tcp://192.168.157.4:43091", "tcp://192.168.192.4:42475", "tcp://192.168.208.132:43737"]}, "task": {"type": "worker", "index": 1}}',
 'tcp://192.168.157.4:43091': '{"cluster": {"worker": ["tcp://192.168.122.132:33355", "tcp://192.168.124.4:34529", "tcp://192.168.157.4:43091", "tcp://192.168.192.4:42475", "tcp://192.168.208.132:43737"]}, "task": {"type": "worker", "index": 2}}',
 'tcp://192.168.192.4:42475': '{"cluster": {"worker": ["tcp://192.168.122.132:33355", "tcp://192.168.124.4:34529", "tcp://192.168.157.4:43091", "tcp://192.168.192.4:42475", "tcp://192.168.208.132:43737"]}, "task": {"type": "worker", "index": 3}}',
 'tcp:

In [26]:
model_params = {'n_epochs': 5, 
                'base_lr': .02,
               'batchsize': 32}

wbargs = {**model_params,
    'classes':10,
    'Notes':"tf_mm",
    'Tags': ['multimachine', 'gpu', 'tensorflow'],
    'dataset':"Clothing",
    'architecture':"ResNet"}


In [27]:
%%time

futures = client.run(train_model_fit, **model_params)
futures_gathered = client.gather(futures)
futures_computed = client.compute(futures_gathered, sync=False)

import logging

results = []
errors = []
for fut in futures_computed:
    try:
        result = fut.result()
    except Exception as e:
        errors.append(e)
        logging.error(e)
    else:
        results.extend(result)

UnknownError: Could not start gRPC server